In [121]:
from urllib.request import urlopen
import json
import pandas as pd
pd.set_option('display.max_rows', 500)

In [71]:
input_string = str(input('GP; Year; Session; Seperate with ;'))
gp = input_string.split(";")[0].strip()
year = input_string.split(";")[1].strip()
session = input_string.split(";")[2].strip()
print(gp, year, session)

GP; Year; Session; Seperate with ;Silverstone; 2024; Race
Silverstone 2024 Race


In [88]:
response = urlopen(f'https://api.openf1.org/v1/sessions?location={gp}&year={year}&session_name={session}')
data = json.loads(response.read().decode('utf-8'))
session_key = data[0]['session_key']
print(session_key)

9558


In [143]:
response = urlopen(f'https://api.openf1.org/v1/drivers?session_key={session_key}')
drivers = json.loads(response.read().decode('utf-8'))
grid = pd.json_normalize(drivers)
grid = grid[['session_key', 'team_name', 'name_acronym', 'driver_number', 'broadcast_name']]
'''position data'''
response = urlopen(f'https://api.openf1.org/v1/position?session_key={session_key}')
pos_response = json.loads(response.read().decode('utf-8'))
pos = pd.json_normalize(pos_response)
pos['date'] = pd.to_datetime(pos['date'])
idx = pos.groupby('driver_number')['date'].idxmax()
pos = pos.loc[idx]
pos = pos.sort_values(by=['position'], ascending=True)
pos = pos[['driver_number', 'position']]
grid = pd.merge(grid, pos, on=['driver_number'], how='left', suffixes=(None, '_right1'))
'''lap data'''
response = urlopen(f'https://api.openf1.org/v1/laps?session_key={session_key}')
lap_response = json.loads(response.read().decode('utf-8'))
lap = pd.json_normalize(lap_response)
lap['date_start'] = pd.to_datetime(lap['date_start'])
idx = lap.groupby('driver_number')['date_start'].idxmax()
lap = lap.loc[idx]
lap = lap[['session_key', 'driver_number', 'i1_speed', 'i2_speed',
           'st_speed', 'date_start', 'lap_duration', 'is_pit_out_lap',
           'duration_sector_1', 'duration_sector_2', 'duration_sector_3',
           'segments_sector_1', 'segments_sector_2', 'segments_sector_3',
           'lap_number']]
grid = pd.merge(grid, lap, on=['driver_number'], how='left', suffixes=(None, '_right2'))
grid = grid.sort_values(by=['position'], ascending=True)


grid

,session_key,team_name,name_acronym,driver_number,broadcast_name,position,session_key_right2,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
15,9558,Mercedes,HAM,44,L HAMILTON,1,9558.0,294.0,264.0,301.0,2024-07-07 15:24:09.955000+00:00,90.146,False,29.302,36.135,24.709,"[None, 2049, 2049, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
0,9558,Red Bull Racing,VER,1,M VERSTAPPEN,2,9558.0,298.0,266.0,306.0,2024-07-07 15:24:12.328000+00:00,89.089,False,29.149,35.714,24.226,"[None, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
3,9558,McLaren,NOR,4,L NORRIS,3,9558.0,298.0,262.0,304.0,2024-07-07 15:24:16.709000+00:00,90.913,False,29.626,36.650,24.637,"[None, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
19,9558,McLaren,PIA,81,O PIASTRI,4,9558.0,294.0,265.0,302.0,2024-07-07 15:24:22.700000+00:00,89.928,False,29.094,35.999,24.835,"[2049, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2049, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
16,9558,Ferrari,SAI,55,C SAINZ,5,9558.0,299.0,264.0,301.0,2024-07-07 15:24:59.135000+00:00,88.293,False,28.550,35.619,24.124,"[2048, 2048, 2051, 2048, 2048, 2049, 2048]","[2049, 2051, 2048, 2049, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",51.0
13,9558,Haas F1 Team,HUL,27,N HULKENBERG,6,9558.0,288.0,261.0,298.0,2024-07-07 15:25:04.989000+00:00,90.847,False,29.432,36.372,25.043,"[2049, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2051, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
8,9558,Aston Martin,STR,18,L STROLL,7,9558.0,301.0,262.0,308.0,2024-07-07 15:25:06.631000+00:00,90.053,False,29.025,36.298,24.730,"[None, 2049, 2049, 2048, 2048, 2048, 2048]","[2049, 2048, 2048, 2048, 2049, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
6,9558,Aston Martin,ALO,14,F ALONSO,8,9558.0,283.0,261.0,300.0,2024-07-07 15:25:12.203000+00:00,91.430,False,29.311,37.163,24.956,"[None, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
11,9558,Williams,ALB,23,A ALBON,9,9558.0,295.0,263.0,307.0,2024-07-07 15:25:18.725000+00:00,89.718,False,29.121,36.186,24.411,"[None, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
10,9558,RB,TSU,22,Y TSUNODA,10,9558.0,298.0,260.0,298.0,2024-07-07 15:25:27.234000+00:00,92.090,False,29.552,36.953,25.585,"[None, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2064, 2064, 2064]",52.0
